In [12]:
from sys import version
print(version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [ ]:
#from PyQt5 import QtCore, QtGui, QtWidgets
#from PyQt5.QtWidgets import QFileDialog, QMessageBox
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tifffile import imread, imshow
from scipy.optimize import curve_fit

#Get all images
from PIL import Image, ImageOps
import glob
image_list = []
a=glob.glob('C:\\Users\\Desktop\\GIWAXS\\*.tif')
a.sort()
for filename in a:
    im=Image.open(filename)   
    image_list.append(im)
    print(image_list[-1].filename)  
    
Image_no=len(image_list)
    
#im2=image_list[-1]
#im.show()
   
print(len(image_list))

c_d = 1
if c_d==1: #film
    cen_x = float(1475-1395)
    cen_y = float(1549)
    sdd = float(416.97)
    xlam = 12.3984/float(10)
    pix_size = float(0.171)
    inc_ang = (np.pi/180)*float(0.2)
    k = 2*np.pi/xlam
    qxy_max = float(2.5)
    qz_max = float(2.5)
    int_max = float(800)
    plot_xlim = float(3.0)
elif c_d==2: 
    print('case not defined')




In [68]:
img = imread(image_list[0].filename)
print(img.shape)
### Draw sector mask
from skimage import data, draw
# --- coordinate specification
r0, c0 = int(cen_y), int(cen_x)  # circle center (row, column)
#print(r0, c0)
R1 = 1800  # circle outer radius
R2 = 80  #circle inner radius
theta0 = np.deg2rad(3)  # angle #1 for arc
theta1 = np.deg2rad(90)  # angle #2 for arc
# Above, I provide two angles, but you can also just give the two
# coordinates below directly
r1, c1 = r0 - 1.5 * R1 * np.sin(theta0), c0 + 1.5 * R1 * np.cos(theta0)  # arc coord #1
r2, c2 = r0 - 1.5 * R1 * np.sin(theta1), c0 + 1.5 * R1 * np.cos(theta1)  # arc coord #2

# --- mask calculation
mask_circle = np.zeros(img.shape[:2], dtype=bool)
mask_poly = np.zeros(img.shape[:2], dtype=bool)
rr, cc = draw.disk((r0, c0), R1, shape=mask_circle.shape)
mask_circle[rr, cc] = 1
rr, cc = draw.disk((r0, c0), R2, shape=mask_circle.shape)
mask_circle[rr, cc] = 0
rr, cc = draw.polygon([r0, r1, r2, r0],
                      [c0, c1, c2, c0], shape=mask_poly.shape)
mask_poly[rr, cc] = 1
mask_sector = mask_circle & mask_poly

(1679, 1475)


In [ ]:

#Total stack spectra going to make: 10
kk = 2*len(image_list)
final_data = np.empty(kk, dtype=object)
i_im_count = 0
Blade_corr = 0
#ITO correction: 0 to skip, 1 to go ahead
ITO_correction = 0
#Intensity correction: 0 to skip, 1 to go ahead
Intensity_correction = 0
Intensity_parameter=0 #q range for intensity normalisation
def func(x, a, x0, sigma):
    return y0+a*np.exp(-(x-x0)**2/(2*sigma**2))

for i_im in range(0,int(kk/2),3):  #range(0,19,1)
    i_im_count += 1
    # Get image filename
    img_0 = imread(image_list[i_im].filename)
    im_filename=os.path.basename(image_list[i_im].filename).split('.')[0]
    print(im_filename)
    # Get image parameters
    img=np.fliplr(img_0)
    a = img.shape[0]
    b = img.shape[1]

    if Blade_corr==1:
        G=np.max(img, axis=1)
        G_len=len(G)
        G1=np.zeros([1,1679])
        k1=np.zeros(1649)
        for i3 in range(0, 1649):
            temp=img[i3][:]
            k1[i3]=np.percentile(temp, 20)
        #print(k1)
        k0=range(1649)
        bg_x=[]
        bg_y=[]
        k_start = 1
        for ki in k0:
            if k1[ki] > k_start:
                bg_x.append(k0[ki])
                bg_y.append(k1[ki])

        for i3 in range(len(bg_x)):
            i4=int(bg_x[i3])
            temp=img[i4][:]
            temp=temp/bg_y[i3]*k_start*50
            img[i4][:]=temp

    
    # Create new image
    img2 = np.concatenate((img[:, 0:int(cen_x + 1)], np.full((a, 1), 0), img[:, int(cen_x):]), axis=1)
    

    
    # Create meshgrids for X and Z pixels
    X, Z = np.meshgrid(np.arange(b + 3) - int(cen_x + 1), np.arange(a + 1) - int(cen_y))
    X[:,int(cen_x + 2):] -= 1
    
    # Calculate angles for q-maps
    L = np.sqrt(np.square(pix_size*X) + sdd**2)
    theta = np.arctan(pix_size*np.divide(Z, L))
    phi = np.arctan(pix_size*X/sdd)
    
    # Calculate q-maps
    q_x = k*np.multiply(np.cos(theta), np.sin(phi))
    q_y = k*np.multiply(np.cos(theta), np.cos(phi)) - k*np.cos(inc_ang)
    q_z = -k*np.sin(theta) + k*np.sin(inc_ang)
    
    # Calculate q_r
    q_r = np.sqrt(np.square(q_x) + np.square(q_y))
    q_r[:, 0:int(cen_x + 2)] = -1*q_r[:, 0:int(cen_x + 2)]
    
    # Get plot parameters
    qxy_min = float(0)
    #qxy_max = float(1.5)
    qz_min = float(0)
    #qz_max = float(1.5)
    int_min = float(0)
    int_max = float(400)
    font_name = 'Arial'
    font_size = int(8)
    log_color = False
    colormap = 'viridis'
    tick_major = qxy_max/5
    tick_minor = qxy_max/10
    
    if log_color:
        img2 = np.ma.log(img2)
        
        if int_min != 0:
            int_min = np.log(int_min)
    
        if int_max != 0:
            int_max = np.log(int_max)
    
    
    # Create figures
    mpl.rcParams['font.family'] = font_name
    plt.rcParams['axes.linewidth'] = 1
    plt.rcParams['font.size'] = font_size
    
    # In case of different aspect ratios
    ratio = (qxy_max - qxy_min)/(qz_max - qz_min)
    
    # Generate figure and subplot
    plt.ion()
    fig = plt.figure(figsize=(ratio*2.4+0.6, ratio*2.4), tight_layout=True, dpi=300)
    ax = plt.subplot(111)
    
    # Set tick parameters
    ax.xaxis.set_tick_params(size=5, width=1, direction='out')
    ax.xaxis.set_tick_params(which='minor', size=3, width=1, direction='out')
    ax.yaxis.set_tick_params(size=5, width=1, direction='out')
    ax.yaxis.set_tick_params(which='minor', size=3, width=1, direction='out')
    
    pcm=ax.pcolormesh(q_r, q_z, img2, vmin=int_min, vmax=int_max, cmap=colormap, shading='flat')
    fig.colorbar(pcm, ax=ax)
    #print(q_z)
    #print(q_r)
    #print(img2)
    #print(np.shape(q_z))
    #print(np.shape(q_r))
    #print(np.shape(img2))
    

    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(tick_major))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(tick_major))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(tick_minor))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(tick_minor))
    
    ax.set_xlabel(u'q$\mathregular{_{xy}}$ (\u00c5$\mathregular{^{-1}}$)')
    ax.set_ylabel(u'q$\mathregular{_{z}}$ (\u00c5$\mathregular{^{-1}}$)')
    
    ax.set_xlim(qxy_min, qxy_max)
    ax.set_ylim(qz_min, qz_max)
    
    saveFileName = Path(image_list[i_im].filename)
    saveFileName = str(saveFileName.parent) + '/' + str(saveFileName.stem) + '_GIXS.png'
    
    plt.savefig(saveFileName)
    plt.show()
    
    #1D plot
    # 
    
    

    
